In [28]:
################################
########### IMPORTS ############
################################

In [29]:
from pycbc import distributions
from pycbc.waveform import get_td_waveform, td_approximants
from pycbc.detector import Detector
import matplotlib.pyplot as plt
import numpy as np
import gwpy
import pylab
from tqdm.notebook import tqdm
from gwpy.timeseries import TimeSeries
import pandas as pd
import os
import csv
import pycbc.noise
import pycbc.psd
from pycbc.filter import matched_filter

In [30]:
print("Generated Binary Mass Distributions for BBH")
bbh_two_mass_distributions = distributions.Uniform(mass1=(10, 50),
                                               mass2=(10, 50))

bbh_two_mass_samples = bbh_two_mass_distributions.rvs(size=1000)

print("Generated Binary Mass Distributions for BNS")
bns_two_mass_distributions = distributions.Uniform(mass1=(1, 2),
                                               mass2=(1, 2))

bns_two_mass_samples = bns_two_mass_distributions.rvs(size=1000)

Generated Binary Mass Distributions for BBH
Generated Binary Mass Distributions for BNS


In [ ]:
for i in tqdm(range(1000)):
    
    # The color of the noise matches a PSD which you provide
    flow = 30.0
    delta_f = 1.0 / 16
    flen = int(2048 / delta_f) + 1
    psd = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, flow)

    # Generate 4 seconds of noise at 4096 Hz
    delta_t = 1.0 / 4096
    tsamples = int(4 / delta_t)
    noise = pycbc.noise.noise_from_psd(tsamples, delta_t, psd)

    noise *= 1e21
    noise *= 0.4
    noise = TimeSeries.from_pycbc(noise)

    noise *= 1e-17
    noise.write("Downloads/Gravitational-Wave-Detection-Using-Deep-Learning/raw_val_data_files/noise_templates/noise_4k_"+str(i)+".txt")

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
start_times_bbh = [0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5]
start_times_bns = [0, 0.5, 1, 1.5, 2, 2.5, 3]

for i in tqdm(range(len(bbh_two_mass_samples))):

    hp1, hc1 = get_td_waveform(approximant="SEOBNRv2",                                
                         mass1=bbh_two_mass_samples[i][0],
                         mass2=bbh_two_mass_samples[i][1],
                         delta_t=1.0/4096,
                         f_lower=40)


    hp2, hc2 = get_td_waveform(approximant="IMRPhenomPv2_NRTidal", 
                         mass1=bns_two_mass_samples[i][0],
                         mass2=bns_two_mass_samples[i][1],
                         delta_t=1.0/4096,
                         f_lower=40)


    bbh_signal = TimeSeries.from_pycbc(hp1)
    st1 = np.random.randint(0, 8)
    bbh_signal.t0 = start_times_bbh[st1]
    bbh_signal = (bbh_signal/(max(bbh_signal.max(), np.abs(bbh_signal.min()))))*0.2


    # Extract the last 1 sec from the BNS signal
    t = hp2.get_end_time()
    hp2 = hp2.time_slice(t-1, t)

    bns_signal = TimeSeries.from_pycbc(hp2)
    st2 = np.random.randint(0, 7)
    bns_signal.t0 = start_times_bns[st2]
    bns_signal = bns_signal.taper()
    bns_signal = (bns_signal/(max(bns_signal.max(), np.abs(bns_signal.min()))))*0.2

    # The color of the noise matches a PSD which you provide
    flow = 30.0
    delta_f = 1.0 / 16
    flen = int(2048 / delta_f) + 1
    psd = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, flow)

    # Generate 4 seconds of noise at 4096 Hz
    delta_t = 1.0 / 4096
    tsamples = int(4 / delta_t)
    noise = pycbc.noise.noise_from_psd(tsamples, delta_t, psd)

    noise *= 1e21
    noise *= 0.4
    noise = TimeSeries.from_pycbc(noise)


    data1 = noise.inject(bbh_signal)
    data2 = noise.inject(bns_signal)

    data1 *= 1e-17
    data2 *= 1e-17
    
data1.write("Downloads/Gravitational-Wave-Detection-Using-Deep-Learning/raw_val_data_files/merged_bbh_noise_signal/merged_noise_signal_"+str(i)+".txt")
data2.write("Downloads/Gravitational-Wave-Detection-Using-Deep-Learning/raw_val_data_files/merged_bns_noise_signal/bns_merged_noise_signal_"+str(i)+".txt")


  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
# # Alternate

# # Not needed since the above method worked.
# start_times_bbh = [0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5]
# start_times_bns = [0, 0.5, 1, 1.5, 2, 2.5, 3]

# for i in tqdm(range(1000)):

#     bbh_signal = TimeSeries.read(" Downloads/Gravitational-Wave-Detection-Using-Deep-Learning/raw_val_data_files/bbh_signal/bbh_4k_"+str(i)+".txt")
#     st1 = np.random.randint(0, 8)
#     bbh_signal.t0 = start_times_bbh[st1]
#     bbh_signal = (bbh_signal/(max(bbh_signal.max(), np.abs(bbh_signal.min()))))*0.2

#     bns_signal = TimeSeries.read(" Downloads/Gravitational-Wave-Detection-Using-Deep-Learning/raw_val_data_files/bns_signal/bns_signal_"+str(i)+".txt")
#     st2 = np.random.randint(0, 7)
#     bns_signal.t0 = start_times_bns[st2]
#     bns_signal = bns_signal.taper()
#     bns_signal = (bns_signal/(max(bns_signal.max(), np.abs(bns_signal.min()))))*0.2

#     # The color of the noise matches a PSD which you provide
#     flow = 30.0
#     delta_f = 1.0 / 16
#     flen = int(2048 / delta_f) + 1
#     psd = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, flow)

#     # Generate 4 seconds of noise at 4096 Hz
#     delta_t = 1.0 / 4096
#     tsamples = int(4 / delta_t)
#     noise = pycbc.noise.noise_from_psd(tsamples, delta_t, psd)

#     noise *= 1e21
#     noise *= 0.4
#     noise = TimeSeries.from_pycbc(noise)

#     data1 = noise.inject(bbh_signal)
#     data2 = noise.inject(bns_signal)

#     data1 *= 1e-17
#     data2 *= 1e-17

#     data1.write(" Downloads/Gravitational-Wave-Detection-Using-Deep-Learning/raw_val_data_files/merged_bbh_noise_signal/merged_noise_signal_"+str(i)+".txt")
#     data2.write(" Downloads/Gravitational-Wave-Detection-Using-Deep-Learning/raw_val_data_files/merged_bns_noise_signal/bns_merged_noise_signal_"+str(i)+".txt")

In [ ]:
# Merging BBH Noise + Signal Templates into single csv file

import os
import csv
import pandas as pd
from tqdm import tqdm

# Define directory path
path = "Downloads/Gravitational-Wave-Detection-Using-Deep-Learning/raw_val_data_files/merged_bbh_noise_signal/"
output_file = " Downloads/Gravitational-Wave-Detection-Using-Deep-Learning/raw_val_data_files/val_Final_BBH_Merged_Noise_Signal_Reduced_No_ABS.csv"

# Ensure output file is not a directory
if os.path.isdir(output_file):
    raise IsADirectoryError(f"Expected a file, but found a directory: {output_file}")

# Get only valid data files, ignoring hidden files like .DS_Store
files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and not f.startswith('.')]

# Open the output CSV file
with open(output_file, 'w', newline='') as f:
    cw = csv.writer(f)

    # Iterate through each file
    for i in tqdm(files):
        file_path = os.path.join(path, i)  # Ensure proper file path
        
        try:
            # Attempt to read using UTF-8 encoding first
            df = pd.read_csv(file_path, sep=' ', header=None, encoding='utf-8', engine='c')
        except UnicodeDecodeError:
            try:
                # Try ISO-8859-1 if UTF-8 fails
                df = pd.read_csv(file_path, sep=' ', header=None, encoding='ISO-8859-1', engine='c')
            except Exception as e:
                print(f"❌ Error reading {file_path}: {e}")
                continue  # Skip file if all attempts fail
        
        # Ensure at least 2 columns exist before writing
        if df.shape[1] > 1:
            c = df.iloc[:, 1]  # Select the second column correctly
            cw.writerow(c)

print(f"✅ Merging complete. Output saved to: {output_file}")

100%|██████████| 2000/2000 [00:32<00:00, 60.98it/s]

✅ Merging complete. Output saved to: /Users/adamvanzant/Downloads/Gravitational-Wave-Detection-Using-Deep-Learning/raw_val_data_files/val_Final_BBH_Merged_Noise_Signal_Reduced_No_ABS.csv


In [ ]:
# Merging Noise Templates into single csv file
import os
import csv
import pandas as pd
from tqdm import tqdm

# Define paths
path_1 = " Downloads/Gravitational-Wave-Detection-Using-Deep-Learning/raw_val_data_files/noise_templates/"
output_file = " Downloads/Gravitational-Wave-Detection-Using-Deep-Learning/raw_val_data_files/val_Final_Merged_Noise_Reduced_No_Abs.csv"

# Ensure output file is not a directory
if os.path.isdir(output_file):
    raise IsADirectoryError(f"Expected a file, but found a directory: {output_file}")

# Get only valid files (ignores hidden/system files like .DS_Store)
files_1 = [f for f in os.listdir(path_1) if os.path.isfile(os.path.join(path_1, f)) and not f.startswith('.')]

# Open the output CSV file safely
with open(output_file, 'w', newline='') as f1:
    cw_1 = csv.writer(f1)

    # Process each file
    for i in tqdm(files_1):
        file_path = os.path.join(path_1, i)  # Ensure proper path formatting

        try:
            # Attempt to read using UTF-8 encoding first
            df = pd.read_csv(file_path, sep=' ', header=None, encoding="utf-8", engine="c")
        except UnicodeDecodeError:
            try:
                # If UTF-8 fails, try ISO-8859-1 as fallback
                df = pd.read_csv(file_path, sep=' ', header=None, encoding="ISO-8859-1", engine="c")
            except Exception as e:
                print(f"❌ Error reading {file_path}: {e}")
                continue  # Skip this file if all attempts fail

        # Ensure at least 2 columns exist before writing
        if df.shape[1] > 1:
            c = df.iloc[:, 1]  # Select second column
            cw_1.writerow(c)

print(f"✅ Merging complete. Output saved to: {output_file}")

100%|██████████| 1000/1000 [00:16<00:00, 61.25it/s]

✅ Merging complete. Output saved to: /Users/adamvanzant/Downloads/Gravitational-Wave-Detection-Using-Deep-Learning/raw_val_data_files/val_Final_Merged_Noise_Reduced_No_Abs.csv


In [ ]:
# Merging BNS Noise + Signal Templates into single csv file

import os
import csv
import pandas as pd
from tqdm import tqdm

# Define paths
path = " Downloads/Gravitational-Wave-Detection-Using-Deep-Learning/raw_val_data_files/merged_bns_noise_signal/"
output_file = "Downloads/Gravitational-Wave-Detection-Using-Deep-Learning/raw_val_data_files/val_Final_BNS_Merged_Noise_Signal_Reduced_No_ABS.csv"

# Ensure the output file is not a directory
if os.path.isdir(output_file):
    raise IsADirectoryError(f"Expected a file, but found a directory: {output_file}")

# Get only valid files (ignores hidden/system files like .DS_Store)
files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and not f.startswith('.')]

# Open the output CSV file safely
with open(output_file, 'w', newline='') as f:
    cw = csv.writer(f)

    # Process each file
    for i in tqdm(files):
        file_path = os.path.join(path, i)  # Ensure correct path

        try:
            # Try reading using UTF-8 first
            df = pd.read_csv(file_path, sep=' ', header=None, encoding="utf-8", engine="c")
        except UnicodeDecodeError:
            try:
                # If UTF-8 fails, try ISO-8859-1
                df = pd.read_csv(file_path, sep=' ', header=None, encoding="ISO-8859-1", engine="c")
            except Exception as e:
                print(f"❌ Error reading {file_path}: {e}")
                continue  # Skip this file if all attempts fail

        # Ensure at least 2 columns exist before writing
        if df.shape[1] > 1:
            c = df.iloc[:, 1]  # Select second column
            cw.writerow(c)

print(f"✅ Merging complete. Output saved to: {output_file}")

100%|██████████| 1000/1000 [00:16<00:00, 62.36it/s]

✅ Merging complete. Output saved to: /Users/adamvanzant/Downloads/Gravitational-Wave-Detection-Using-Deep-Learning/raw_val_data_files/val_Final_BNS_Merged_Noise_Signal_Reduced_No_ABS.csv
